In [70]:
import numpy as np
import pandas as pd
from scipy.optimize import nnls

In [79]:
path="C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/"#proportion_pxt_neh/"
time=6
pw=0.046

In [80]:
data=pd.read_csv(path+"__CPSM_MOUSE.csv")
data=data[data.exp_Time==time]
data=data[data.exp_Time==time].reset_index(drop="True")
data.head()

,Peptied,charge,exp_Time,NEH,e_NEH,pxt,e_pxt,mo,m1,m2,...,io,i1,i2,i3,i4,i5,NEH_new,new_error,new_cor,new_rsq
0,IEFEGQSVDFVDPNKQNLIAEVSTK,3,6,47,72,0.046,0.012,0.19687,0.30439,0.25117,...,0.119984,0.206111,0.220466,0.200600,0.148603,0.104237,72,0.108783,-0.218991,-4.914502
1,PSYVLSGSAMNVVFSEDEMKR,3,6,39,109,0.046,0.006,0.24027,0.30714,0.23256,...,0.147801,0.236769,0.233689,0.179064,0.127946,0.074731,26,0.042798,0.759417,0.446833
2,QIDTLAAEYPSVTNYLYVTY,2,6,30,110,0.046,0.006,0.24881,0.33036,0.23518,...,0.149264,0.249506,0.245224,0.183089,0.113240,0.059677,30,0.055926,0.656399,0.327544
3,AVQFHPEVSPGPTDTEYLFDSFFSLIK,3,6,46,110,0.046,0.007,0.15607,0.27736,0.25958,...,0.095067,0.195997,0.230701,0.214391,0.160562,0.103283,1,0.058577,0.728581,-0.253855
4,SMEYcIIEVNAR,2,6,25,119,0.046,0.004,0.40995,0.31550,0.17244,...,0.262964,0.297343,0.229899,0.125343,0.060822,0.023630,21,0.052777,0.860913,0.737709


In [81]:
res=[]
for pep_index in range(data.shape[0]):
    exp_0=list(data.loc[pep_index,["mo","m1","m2","m3","m4","m5"]])
    b=list(data.loc[pep_index,["io","i1","i2","i3","i4","i5"]])#exp_t
    
#     if (data.loc[pep_index,"Peptied"]!="IEFEGQSVDFVDPNK"):
#         continue
    
    A=[]
    for i in range(6):
        A.append(([0]*i)+exp_0[:6-i])
    A=np.array(A).transpose()
#     x=np.matmul( np.linalg.inv ( np.matmul(A.transpose(),A)) , np.matmul(A.transpose(),b)  )
    x,residual=nnls(A,b)
    
#     ypred=[np.dot(A[i],x) for i in range(A.shape[0])]
#     residual=sum([ abs(b[i]-y) for index,y in enumerate(ypred)])
    
    ypred=[np.dot(A[i],x) for i in range(A.shape[0])]
    sigma=sum([ (b[i]-y)**2 for index,y in enumerate(ypred)])/(len(ypred)-1)
    cov=((sigma)*np.linalg.inv(np.matmul(A.transpose(),A)))
    var=[cov[x][x] for x in range(6)]
    
    print("\n",data.loc[pep_index,"Peptied"],data.loc[pep_index,"charge"],"\n",np.corrcoef(x,exp_0)[1][0])
    
    neh= (x[2]/x[1])*2* ((1-pw)/pw) +1
    xm=sum([(index)*val / sum(x) for index,val in enumerate(x)])
    res.append([data.loc[pep_index,"Peptied"],data.loc[pep_index,"charge"],data.loc[pep_index,"NEH"],neh,xm, 
               data.loc[pep_index,"e_NEH"]*data.loc[pep_index,"e_pxt"],residual])
    
    r=x[1]/x[2]
    c=data.loc[pep_index,"e_NEH"]*data.loc[pep_index,"e_pxt"]  #0.0610440961749121
    t_pw= (r*c-2)/ (r-2)
    print(pw,neh,pw*neh,"\t",c,"\t",t_pw,c/t_pw)
#     print(data.loc[pep_index,"Peptied"],x,end="\n\n")


 IEFEGQSVDFVDPNKQNLIAEVSTK 3 
 0.3031255962626352
0.046 72.56741825949669 3.3381012399368477 	 0.8640000745654104 	 1.055491150959786 0.8185763317672083

 PSYVLSGSAMNVVFSEDEMKR 3 
 0.5832909305686679
0.046 26.38945032411211 1.2139147149091571 	 0.6540000564418678 	 2.543063301160159 0.2571701837478876

 QIDTLAAEYPSVTNYLYVTY 2 
 0.630033892351038
0.046 30.093482631749968 1.3843002010604986 	 0.660000056959683 	 1.8440275200621088 0.35791225986554337

 AVQFHPEVSPGPTDTEYLFDSFFSLIK 3 
 0.2647018861720136
0.046 40.91405581367729 1.882046567429155 	 0.770000074990091 	 1.248762295055444 0.6166106055883948

 SMEYcIIEVNAR 2 
 0.9180562256772333
0.046 21.18884648268975 0.9746869382037285 	 0.4760000226087862 	 -18.74853861830544 -0.02538864667265511

 TAVDSGIALLTNFQVTK 2 
 0.803744781846519
0.046 17.776104662573466 0.8177008144783794 	 0.460000021848827 	 -1.8259038990337209 -0.25193002878862447

 EPLFGISTGNIITGLAAGAK 3 
 0.7596320154189345
0.046 34.78970497537874 1.600326428867422 	 0.6300000

C:\Users\hmdebern.UTMB-USERS-M\AppData\Local\Temp\ipykernel_53320\1694189291.py:31: RuntimeWarning: divide by zero encountered in double_scalars
  r=x[1]/x[2]
C:\Users\hmdebern.UTMB-USERS-M\AppData\Local\Temp\ipykernel_53320\1694189291.py:33: RuntimeWarning: invalid value encountered in double_scalars
  t_pw= (r*c-2)/ (r-2)


In [82]:
sum([(index)* (val / sum(x)) for index,val in enumerate(x)])


0.6615735676227261

In [83]:
res=pd.DataFrame(res)
res.columns=["Peptide","Charge","True NEH","e NEH","sum_xi","pxt_neh","residual"]
res["dif"]=abs(res["True NEH"]- res["e NEH"])
# res["test"]=100*abs(res["sum_xi"]-res["pxt_neh"])/res["pxt_neh"]
res["test"]=100*abs(res["sum_xi"]-res["pxt_neh"])/res["pxt_neh"]

res=res.sort_values(by="dif")
print(res.shape)
res[["Peptide","Charge","True NEH","e NEH","dif"]].head(50)

(42, 9)


,Peptide,Charge,True NEH,e NEH,dif
8,TAVDSGIALLTNFQVTK,3,25,24.943721,0.056279
2,QIDTLAAEYPSVTNYLYVTY,2,30,30.093483,0.093483
22,GQILTMANPIIGNGGAPDTTAR,3,43,43.163570,0.163570
33,NVPATPVAWPSQEGQNPSLSSIR,2,52,52.545964,0.545964
36,ETLIDLGTK,2,11,11.760003,0.760003
19,LYFEELSLER,2,20,19.070148,0.929852
17,IEFEGQSVDFVDPNK,3,28,26.975259,1.024741
6,EPLFGISTGNIITGLAAGAK,3,36,34.789705,1.210295
18,QAFITAQNHGY,2,24,22.466045,1.533955
16,FVHDNYVIR,2,12,13.716656,1.716656


In [84]:
res[["Peptide","Charge","True NEH","e NEH","dif"]].tail(30)

,Peptide,Charge,True NEH,e NEH,dif
31,TFEESFQK,2,15,12.488094,2.511906
26,NVVFSEDEMKR,2,20,17.032203,2.967797
34,SIFSAVLDELK,2,18,14.701035,3.298965
29,IAPSFAVESMEDALK,2,33,29.671890,3.328110
4,SMEYcIIEVNAR,2,25,21.188846,3.811154
21,IASVQTNEVGLK,2,21,16.979030,4.020970
40,YMESDGIK,2,13,8.519164,4.480836
3,AVQFHPEVSPGPTDTEYLFDSFFSLIK,3,46,40.914056,5.085944
27,GNDVLVIEcNLR,2,20,14.869769,5.130231
20,VLGTSVESIMATEDR,2,28,33.246897,5.246897


In [85]:
res.to_csv("testx2x1.csv",index=False)

In [78]:
# temp=pd.read_csv("C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/pw_known_search_neh/__CPSM_MOUSE.csv")
# temp["dif"]=temp.NEH-temp.e_NEH
# temp.sort_values(by="Peptied") 